In [ ]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt

import glob

path_list = glob.glob('./natural_grape_masks_*/*.png')

def if_same_array(px, py):
    num_mutched_elements = np.count_nonzero(px==py)
    if (num_mutched_elements== px.size and num_mutched_elements== py.size):
        return True
    else:
        return False
   
for label_path in path_list:
    img = cv2.imread(label_path)

    pixel_dict = {'background':np.array([0, 0, 0]), 'stem':np.array([255, 255, 255])}

    count_dict_length = 0
    idx_cnt = 0
    berry_idx = 'id_' + str(idx_cnt)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in pixel_dict:
                if if_same_array(img[i][j], pixel_dict[k]):
                    count_dict_length = 0
                    flag = True
                    break
                else:
                    count_dict_length = count_dict_length + 1
            
            if flag:
                flag = False
                continue
            
            if(count_dict_length == len(pixel_dict)):
                #print(img[i][j])
                pixel_dict[berry_idx] = img[i][j]
                idx_cnt = idx_cnt + 1
                berry_idx = 'id_' + str(idx_cnt)
                

    berry_mask_dict = {}
    index_keys = [k for k in pixel_dict if 'id_' in k]
    idx_counter=0

    for k in index_keys:
        mask_coordinate = np.zeros((0, 2))
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if(if_same_array(img[i][j], pixel_dict[k])):
                    mask_coordinate = np.r_[mask_coordinate, np.array([[i, j]])]
    
        berry_mask_dict['id_' + str(idx_counter)] = mask_coordinate
        idx_counter = idx_counter + 1
                  
                
    for k in berry_mask_dict:
        berry_mask_dict[k] = berry_mask_dict[k].astype(int)

    pil_mask = np.zeros(img.shape)
    for i, key in enumerate(berry_mask_dict):
        for idx in range(len(berry_mask_dict[key])):
            pil_mask[berry_mask_dict[key][idx][0]][berry_mask_dict[key][idx][1]] = int(i + 1)
        
    pil_mask = pil_mask.astype(np.uint8)
    pil_mask = Image.fromarray(pil_mask)
    pil_mask.save(label_path)
    print(label_path)


./natural_grape_masks_test/before_fuji226_02_label.png


In [43]:
import glob

path_list = glob.glob('./natural_grape_masks_*/*.png')

In [44]:
len(path_list)

497

In [38]:
!pwd

/Users/tamurataito/Documents/grape_dataset


In [7]:
berry_mask_dict.keys()

dict_keys(['id_0', 'id_1', 'id_2', 'id_3', 'id_4', 'id_5', 'id_6', 'id_7', 'id_8', 'id_9', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28'])

In [8]:
import PIL
from PIL import Image

In [12]:
img.shape

(960, 960)

In [14]:
berry_mask_dict['id_0']

array([[183, 363],
       [183, 364],
       [183, 365],
       ...,
       [280, 377],
       [280, 378],
       [280, 379]])

In [35]:
pil_mask = np.zeros(img.shape)
for i, key in enumerate(berry_mask_dict):
    for idx in range(len(berry_mask_dict[key])):
        pil_mask[berry_mask_dict[key][idx][0]][berry_mask_dict[key][idx][1]] = int(i + 1)
        
pil_mask = pil_mask.astype(np.uint8)
pil_mask = Image.fromarray(pil_mask)
im.save('./test_pil_mask.png')

In [29]:
Image.save(pil_mask, './test_pil_mask.png')

AttributeError: module 'PIL.Image' has no attribute 'save'

In [36]:
np.unique(pil_mask)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=uint8)

In [32]:
pil_mask.shape

(960, 960)

In [34]:
im = Image.fromarray(pil_mask)
im.save('./test_pil_mask.png')


In [9]:
!ls

JPEGImages_test                    make_PIL_masks.ipynb
JPEGImages_train                   natural_grape_masks_test
Untitled.ipynb                     natural_grape_masks_train
label_7_37.png                     natural_grape_masksのコピー


In [4]:
grey_img = cv2.imread(label_path, 0)
temp_img = np.zeros(grey_img.shape)

hough_threshold = 16

counter=0


for k in berry_mask_dict:
    
    empty_img = temp_img.copy()
    for l in range(len(berry_mask_dict[k])):
        empty_img[berry_mask_dict[k][l][0]][berry_mask_dict[k][l][1]] = 255
    empty_img = empty_img.astype(np.uint8)
         
    # 実ごとのバウンディングボックスを計算する箇所
    ret,thresh = cv2.threshold(empty_img,127,255,0)
    contours,hierarchy = cv2.findContours(thresh, 1, 2)
    cnt = contours[0]
    x,y,w,h = cv2.boundingRect(cnt)
    
    # 実ごとにハフ変換で円を検出してtruncatedか否かを判断する箇所
    img = cv2.medianBlur(empty_img, 5)
    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                            param1=20,param2=hough_threshold,minRadius=0,maxRadius=0)
    
    if not (circles is None):
        truncated = False
    else:
        truncated = True
        
    # .xml ファイルに書き込むアノテーションの情報を追加

    counter = counter + 1
    
#writer.save('./label_7_37.xml')